# Fontainebleau ML Models - Model Registry

This notebook trains ML models for the Fontainebleau Intelligence Agent:
- **Guest Satisfaction Prediction** - Predict guest satisfaction sentiment from feedback features
- **Room Occupancy Forecast** - Predict future room occupancy rates
- **Service Demand Prediction** - Predict demand for hotel services

All models are registered to Snowflake Model Registry and can be added as tools to the Intelligence Agent.

## Prerequisites

**Required Packages** (configured automatically):
- `snowflake-ml-python`
- `scikit-learn`
- `xgboost`
- `matplotlib`

**Database Context:**
- **Database:** FONTAINEBLEAU_INTELLIGENCE  
- **Schema:** ANALYTICS  
- **Warehouse:** FONTAINEBLEAU_WH

**Note:** This notebook uses Snowflake Model Registry. Ensure you have appropriate permissions to create and register models.


## Import Required Packages


In [ ]:
# Import Python packages
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Import Snowpark
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark import Window

# Import Snowpark ML
from snowflake.ml.modeling.preprocessing import StandardScaler, OneHotEncoder
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.linear_model import LinearRegression, LogisticRegression
from snowflake.ml.modeling.ensemble import RandomForestClassifier
from snowflake.ml.modeling.metrics import mean_squared_error, mean_absolute_error, accuracy_score, roc_auc_score
from snowflake.ml.registry import Registry

print("✅ Packages imported successfully")


## Connect to Snowflake

Get active session and set context to Fontainebleau database.


In [ ]:
# Get active Snowflake session
session = get_active_session()

# Set context - MUST MATCH 01_database_and_schema.sql
session.use_database('FONTAINEBLEAU_INTELLIGENCE')
session.use_schema('ANALYTICS')
session.use_warehouse('FONTAINEBLEAU_WH')

print(f"✅ Connected - Role: {session.get_current_role()}")
print(f"   Warehouse: {session.get_current_warehouse()}")
print(f"   Database.Schema: {session.get_fully_qualified_current_schema()}")


---
# MODEL 1: Guest Satisfaction Prediction

Predict guest satisfaction sentiment (POSITIVE, NEUTRAL, NEGATIVE) based on reservation and feedback features.


### Prepare Guest Satisfaction Training Data


In [ ]:
# Get guest satisfaction data with features
# COLUMN NAMES VERIFIED against 02_create_tables.sql:
# - GUESTS: loyalty_tier, total_spend, vip_status
# - RESERVATIONS: adults, children, total_room_revenue, booking_channel, nights
# - ROOM_TYPES: room_category
# - GUEST_FEEDBACK: overall_rating (derive sentiment from 1-5 rating)
satisfaction_df = session.sql("""
SELECT
    gf.feedback_id,
    CASE WHEN g.loyalty_tier IN ('GOLD', 'PLATINUM') THEN TRUE ELSE FALSE END AS is_loyalty_member,
    g.total_spend::FLOAT AS guest_total_spend,
    (r.adults + r.children)::FLOAT AS num_guests,
    r.total_room_revenue::FLOAT AS reservation_price,
    r.nights::FLOAT AS stay_duration_days,
    rt.room_category AS room_category,
    r.booking_channel AS booking_source,
    gf.overall_rating::FLOAT AS satisfaction_rating,
    -- Target: Sentiment derived from overall_rating (1-5 scale)
    CASE 
        WHEN gf.overall_rating >= 4 THEN 2  -- POSITIVE
        WHEN gf.overall_rating = 3 THEN 1   -- NEUTRAL
        ELSE 0                               -- NEGATIVE
    END AS sentiment_label
FROM RAW.GUEST_FEEDBACK gf
JOIN RAW.GUESTS g ON gf.guest_id = g.guest_id
JOIN RAW.RESERVATIONS r ON gf.reservation_id = r.reservation_id
JOIN RAW.ROOM_TYPES rt ON r.room_type_id = rt.room_type_id
WHERE gf.feedback_date >= DATEADD('year', -2, CURRENT_DATE())
  AND gf.overall_rating IS NOT NULL
LIMIT 10000
""")

print(f"Guest satisfaction data: {satisfaction_df.count()} feedback entries")
satisfaction_df.show(5)


### Split Data and Train Guest Satisfaction Model


In [ ]:
# Train/test split (80/20)
train_satisfaction, test_satisfaction = satisfaction_df.random_split([0.8, 0.2], seed=42)

# Drop FEEDBACK_ID (not a feature)
train_satisfaction = train_satisfaction.drop("FEEDBACK_ID")
test_satisfaction = test_satisfaction.drop("FEEDBACK_ID")

# Create pipeline with preprocessing and classification
satisfaction_pipeline = Pipeline([
    ("Encoder", OneHotEncoder(
        input_cols=["ROOM_CATEGORY", "BOOKING_SOURCE"],
        output_cols=["ROOM_CATEGORY_ENC", "BOOKING_SOURCE_ENC"],
        drop_input_cols=True,
        handle_unknown="ignore"
    )),
    ("Scaler", StandardScaler(
        input_cols=["GUEST_TOTAL_SPEND", "NUM_GUESTS", "RESERVATION_PRICE", "STAY_DURATION_DAYS", "SATISFACTION_RATING"],
        output_cols=["GUEST_TOTAL_SPEND_SCALED", "NUM_GUESTS_SCALED", "RESERVATION_PRICE_SCALED", "STAY_DURATION_DAYS_SCALED", "SATISFACTION_RATING_SCALED"]
    )),
    ("Classifier", RandomForestClassifier(
        label_cols=["SENTIMENT_LABEL"],
        output_cols=["PREDICTED_SENTIMENT"],
        n_estimators=100,
        max_depth=10
    ))
])

# Train model
satisfaction_pipeline.fit(train_satisfaction)
print("✅ Guest satisfaction prediction model trained")


### Evaluate and Register Guest Satisfaction Model


In [ ]:
# Make predictions on test set
satisfaction_predictions = satisfaction_pipeline.predict(test_satisfaction)

# Calculate accuracy
accuracy = accuracy_score(
    df=satisfaction_predictions, 
    y_true_col_names="SENTIMENT_LABEL", 
    y_pred_col_names="PREDICTED_SENTIMENT"
)

satisfaction_metrics = {"accuracy": round(accuracy, 4)}
print(f"Model metrics: {satisfaction_metrics}")

# Register model
reg = Registry(session)
reg.log_model(
    model=satisfaction_pipeline,
    model_name="GUEST_SATISFACTION_PREDICTOR",
    version_name="V1",
    comment="Predicts guest satisfaction sentiment (POSITIVE/NEUTRAL/NEGATIVE) using Random Forest based on reservation and guest features",
    metrics=satisfaction_metrics
)

print("✅ Guest satisfaction model registered to Model Registry as GUEST_SATISFACTION_PREDICTOR")


---
# MODEL 2: Room Occupancy Forecast

Predict future monthly room occupancy rates based on historical reservation patterns.


### Prepare Room Occupancy Training Data


In [ ]:
# Get monthly occupancy data with features
# COLUMN NAMES VERIFIED against 02_create_tables.sql:
# - RESERVATIONS: reservation_id, guest_id, room_id, check_in_date, nights, 
#                 total_room_revenue, reservation_status
occupancy_df = session.sql("""
SELECT
    DATE_TRUNC('month', r.check_in_date)::DATE AS occupancy_month,
    MONTH(r.check_in_date) AS month_num,
    YEAR(r.check_in_date) AS year_num,
    COUNT(DISTINCT r.reservation_id)::FLOAT AS total_reservations,
    COUNT(DISTINCT r.guest_id)::FLOAT AS unique_guests,
    COUNT(DISTINCT r.room_id)::FLOAT AS rooms_booked,
    AVG(r.nights)::FLOAT AS avg_stay_duration,
    AVG(r.total_room_revenue)::FLOAT AS avg_booking_value,
    -- Occupancy rate as target (rooms booked / total rooms available)
    (COUNT(DISTINCT r.room_id)::FLOAT / (SELECT COUNT(*) FROM RAW.ROOMS)::FLOAT * 100)::FLOAT AS occupancy_rate
FROM RAW.RESERVATIONS r
WHERE r.check_in_date >= DATEADD('month', -24, CURRENT_DATE())
  AND r.reservation_status IN ('CONFIRMED', 'CHECKED_IN', 'CHECKED_OUT')
GROUP BY DATE_TRUNC('month', r.check_in_date), MONTH(r.check_in_date), YEAR(r.check_in_date)
ORDER BY occupancy_month
""")

print(f"Occupancy data: {occupancy_df.count()} months")
occupancy_df.show(5)


### Split Data and Train Occupancy Forecast Model


In [ ]:
# Train/test split (last 6 months for testing)
train_occupancy = occupancy_df.filter(F.col("OCCUPANCY_MONTH") < F.dateadd("month", F.lit(-6), F.current_date()))
test_occupancy = occupancy_df.filter(F.col("OCCUPANCY_MONTH") >= F.dateadd("month", F.lit(-6), F.current_date()))

# Drop OCCUPANCY_MONTH (DATE type not supported in pipeline)
train_occupancy = train_occupancy.drop("OCCUPANCY_MONTH")
test_occupancy = test_occupancy.drop("OCCUPANCY_MONTH")

# Create pipeline
occupancy_pipeline = Pipeline([
    ("Scaler", StandardScaler(
        input_cols=["MONTH_NUM", "YEAR_NUM", "TOTAL_RESERVATIONS", "UNIQUE_GUESTS", "ROOMS_BOOKED", "AVG_STAY_DURATION", "AVG_BOOKING_VALUE"],
        output_cols=["MONTH_NUM_SCALED", "YEAR_NUM_SCALED", "TOTAL_RESERVATIONS_SCALED", "UNIQUE_GUESTS_SCALED", "ROOMS_BOOKED_SCALED", "AVG_STAY_DURATION_SCALED", "AVG_BOOKING_VALUE_SCALED"]
    )),
    ("LinearRegression", LinearRegression(
        label_cols=["OCCUPANCY_RATE"],
        output_cols=["PREDICTED_OCCUPANCY_RATE"]
    ))
])

# Train model
occupancy_pipeline.fit(train_occupancy)
print("✅ Room occupancy forecasting model trained")


### Evaluate and Register Occupancy Model


In [ ]:
# Make predictions on test set
occupancy_predictions = occupancy_pipeline.predict(test_occupancy)

# Calculate metrics
mae = mean_absolute_error(
    df=occupancy_predictions, 
    y_true_col_names="OCCUPANCY_RATE", 
    y_pred_col_names="PREDICTED_OCCUPANCY_RATE"
)
mse = mean_squared_error(
    df=occupancy_predictions, 
    y_true_col_names="OCCUPANCY_RATE", 
    y_pred_col_names="PREDICTED_OCCUPANCY_RATE"
)
rmse = mse ** 0.5

occupancy_metrics = {"mae": round(mae, 2), "rmse": round(rmse, 2)}
print(f"Model metrics: {occupancy_metrics}")

# Register model
reg.log_model(
    model=occupancy_pipeline,
    model_name="ROOM_OCCUPANCY_FORECASTER",
    version_name="V1",
    comment="Predicts monthly room occupancy rates based on historical reservation patterns using Linear Regression",
    metrics=occupancy_metrics
)

print("✅ Room occupancy model registered to Model Registry as ROOM_OCCUPANCY_FORECASTER")


---
# MODEL 3: Spa Demand Prediction

Predict demand for spa appointments based on historical patterns using SPA_APPOINTMENTS table.


### Prepare Spa Demand Training Data


In [ ]:
# Get daily spa demand data with features
# COLUMN NAMES VERIFIED against 02_create_tables.sql:
# - SPA_APPOINTMENTS: appointment_id, appointment_date, total_amount, appointment_status
spa_demand_df = session.sql("""
SELECT
    sa.appointment_date AS appointment_day,
    DAYOFWEEK(sa.appointment_date)::FLOAT AS day_of_week,
    MONTH(sa.appointment_date) AS month_num,
    -- Daily aggregates
    COUNT(DISTINCT sa.appointment_id)::FLOAT AS appointment_count,
    AVG(sa.total_amount)::FLOAT AS avg_appointment_amount,
    SUM(sa.total_amount)::FLOAT AS total_daily_revenue,
    -- Demand classification (high demand = more than 20 appointments)
    CASE WHEN COUNT(DISTINCT sa.appointment_id) > 20 THEN 1 ELSE 0 END AS high_demand
FROM RAW.SPA_APPOINTMENTS sa
WHERE sa.appointment_date >= DATEADD('year', -1, CURRENT_DATE())
  AND sa.appointment_status IN ('CONFIRMED', 'COMPLETED')
GROUP BY sa.appointment_date, DAYOFWEEK(sa.appointment_date), MONTH(sa.appointment_date)
ORDER BY appointment_day
""")

print(f"Spa demand data: {spa_demand_df.count()} days")
spa_demand_df.show(5)


### Train Spa Demand Prediction Model


In [ ]:
# Train/test split (80/20)
train_demand, test_demand = spa_demand_df.random_split([0.8, 0.2], seed=42)

# Drop APPOINTMENT_DAY (DATE type not supported in pipeline)
train_demand = train_demand.drop("APPOINTMENT_DAY")
test_demand = test_demand.drop("APPOINTMENT_DAY")

# Create pipeline for predicting appointment count (regression)
demand_pipeline = Pipeline([
    ("Scaler", StandardScaler(
        input_cols=["DAY_OF_WEEK", "MONTH_NUM", "AVG_APPOINTMENT_AMOUNT"],
        output_cols=["DAY_OF_WEEK_SCALED", "MONTH_NUM_SCALED", "AVG_APPOINTMENT_AMOUNT_SCALED"]
    )),
    ("LinearRegression", LinearRegression(
        label_cols=["APPOINTMENT_COUNT"],
        output_cols=["PREDICTED_DEMAND"]
    ))
])

# Train model
demand_pipeline.fit(train_demand)
print("✅ Spa demand prediction model trained")


### Evaluate and Register Spa Demand Model


In [ ]:
# Make predictions on test set
demand_predictions = demand_pipeline.predict(test_demand)

# Calculate metrics
mae = mean_absolute_error(
    df=demand_predictions, 
    y_true_col_names="APPOINTMENT_COUNT", 
    y_pred_col_names="PREDICTED_DEMAND"
)
mse = mean_squared_error(
    df=demand_predictions, 
    y_true_col_names="APPOINTMENT_COUNT", 
    y_pred_col_names="PREDICTED_DEMAND"
)
rmse = mse ** 0.5

demand_metrics = {"mae": round(mae, 2), "rmse": round(rmse, 2)}
print(f"Model metrics: {demand_metrics}")

# Register model
reg.log_model(
    model=demand_pipeline,
    model_name="SPA_DEMAND_PREDICTOR",
    version_name="V1",
    comment="Predicts daily spa appointment demand based on day of week and month using Linear Regression",
    metrics=demand_metrics
)

print("✅ Spa demand model registered to Model Registry as SPA_DEMAND_PREDICTOR")


---
# Verify Models in Registry


In [ ]:
# Show all models in the registry
print("Models in registry:")
reg.show_models()

# Show versions for guest satisfaction model
print("\nGuest Satisfaction Predictor versions:")
reg.get_model("GUEST_SATISFACTION_PREDICTOR").show_versions()

# Show versions for occupancy model  
print("\nRoom Occupancy Forecaster versions:")
reg.get_model("ROOM_OCCUPANCY_FORECASTER").show_versions()

# Show versions for spa demand model
print("\nSpa Demand Predictor versions:")
reg.get_model("SPA_DEMAND_PREDICTOR").show_versions()

print("\n✅ All models registered and ready to add to Intelligence Agent")


---
# Test Model Inference

Test calling each model to make predictions.


In [ ]:
# Test guest satisfaction prediction on sample data
satisfaction_model = reg.get_model("GUEST_SATISFACTION_PREDICTOR").default
sample_satisfaction = satisfaction_df.limit(5).drop("FEEDBACK_ID")
satisfaction_preds = satisfaction_model.run(sample_satisfaction, function_name="predict")
print("Guest Satisfaction predictions:")
satisfaction_preds.select("SENTIMENT_LABEL", "PREDICTED_SENTIMENT").show()

# Test room occupancy forecast on recent data
occupancy_model = reg.get_model("ROOM_OCCUPANCY_FORECASTER").default
recent_occupancy = occupancy_df.limit(3).drop("OCCUPANCY_MONTH")
occupancy_preds = occupancy_model.run(recent_occupancy, function_name="predict")
print("\nRoom Occupancy predictions:")
occupancy_preds.select("OCCUPANCY_RATE", "PREDICTED_OCCUPANCY_RATE").show()

# Test spa demand prediction
demand_model = reg.get_model("SPA_DEMAND_PREDICTOR").default
sample_demand = spa_demand_df.limit(5).drop("APPOINTMENT_DAY")
demand_preds = demand_model.run(sample_demand, function_name="predict")
print("\nSpa Demand predictions:")
demand_preds.select("APPOINTMENT_COUNT", "PREDICTED_DEMAND").show()

print("\n✅ All models tested successfully!")


---
# Next Steps

## Add Models to Intelligence Agent

**Option 1: Using the SQL Script (Easiest)**
Run `sql/agent/08_create_intelligence_agent.sql` which automatically configures all 3 ML models.

**Option 2: Manual Configuration in Snowsight**
1. In Snowsight → AI & ML → Agents → FONTAINEBLEAU_INTELLIGENCE_AGENT
2. Go to Tools → + Add → Function
3. Add each model wrapper procedure:
   - **PREDICT_GUEST_SATISFACTION** (from `sql/ml/07_create_model_wrapper_functions.sql`)
   - **FORECAST_ROOM_OCCUPANCY** (from `sql/ml/07_create_model_wrapper_functions.sql`)
   - **PREDICT_SPA_DEMAND** (from `sql/ml/07_create_model_wrapper_functions.sql`)

## Example Questions for Agent

- "Predict guest satisfaction for PLATINUM tier guests"
- "Forecast room occupancy for the next 3 months"
- "Predict spa appointment demand for next weekend"
- "What is the predicted room occupancy rate for December?"

The models will now be available as tools your agent can use!
